In [70]:
# Import required libraries and dependencies

import pandas as pd
import numpy as np
import os
import json
import requests
from dotenv import load_dotenv
%matplotlib inline
import time
import base64
import hmac
import hashlib

from utils import functions_list
#from utils import api_call_all_tickers

In [71]:
# Load .env enviroment variables into the notebook
load_dotenv()

# Get the API key from the environment variable and store as Python variable
kucoin_api_key = os.getenv("KUCOIN_API_KEY")
kucoin_secret_key = os.getenv("KUCOIN_SECRET_KEY")
kucoin_passphrase = os.getenv("KUCOIN_PASSPHRASE")

In [72]:
#pull user account filled spot trade orders data with kucoin api call format
api_key = kucoin_api_key
api_secret = kucoin_secret_key
api_passphrase = kucoin_passphrase
url = 'https://api.kucoin.com/api/v1/fills?currentPage=1&pageSize=500'
now = int(time.time() * 1000)
str_to_sign = str(now) + 'GET' + '/api/v1/fills?currentPage=1&pageSize=500'
signature = base64.b64encode(
    hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
headers = {
    "KC-API-SIGN": signature,
    "KC-API-TIMESTAMP": str(now),
    "KC-API-KEY": api_key,
    "KC-API-PASSPHRASE": passphrase,
    "KC-API-KEY-VERSION": "2"
}

kucoin_response = requests.request('get', url, headers=headers)
#print(kucoin_response.status_code)
kucoin_order_fills = kucoin_response.json()
#print(json.dumps(kucoin_order_fills, indent=2, sort_keys=True))

In [73]:
#use the kucoin response data to pull specific data, organize and convert to workable DataFrame
items = kucoin_order_fills["data"]["items"][:]
trading_pair = []
price = []
order_type = []
quantity = []
cost_usdt = []
fee_usdt = []     

index = 0
for index in range(len(items)):
    for key in items[index]:
        if key == "symbol":
            trading_pair.append(str(items[index][key]))
        if key == "price":
            price.append(float(items[index][key]))
        if key == "side":
            order_type.append(str(items[index][key]))
        if key == "size":
            quantity.append(float(items[index][key]))  
        if key == "funds":
            cost_usdt.append(float(items[index][key]))
        if key == "fee":
            fee_usdt.append(float(items[index][key]))
        

#df = pd.DataFrame(list(zip(trading_pair, price, order_type, quantity, cost_usdt, fee_usdt)

trading_pair_dict = {"Trading Pair":trading_pair}
trading_pair_df = pd.DataFrame(trading_pair_dict)

price_dict = {"Price":price}
price_df = pd.DataFrame(price_dict)

order_type_dict = {"Order Type":order_type}
order_type_df = pd.DataFrame(order_type_dict)

quantity_dict = {"Quantity":quantity}
quantity_df = pd.DataFrame(quantity_dict)

cost_usdt_dict = {"Cost USDT":cost_usdt}
cost_usdt_df = pd.DataFrame(cost_usdt_dict)

fee_usdt_dict = {"Fee USDT":fee_usdt}
fee_usdt_df = pd.DataFrame(fee_usdt_dict)

header = ["Trading Pair", "Price", "Order Type", "Quantity", "Cost USDT", "Fee USDT"]

# Create complete DataFrame 
kucoin_data_past_seven_days_df = trading_pair_df\
.join(price_df, how='left')\
.join(order_type_df, how='left')\
.join(quantity_df, how='left')\
.join(cost_usdt_df, how='left')\
.join(fee_usdt_df, how='left')
    

# Display complete DataFrame
#display(kucoin_data_past_seven_days_df)
#display(kucoin_data_past_seven_days_df.describe())



In [74]:
sum_all_data_df = kucoin_data_past_seven_days_df.groupby(["Trading Pair", "Order Type"], as_index=False).sum()

#display(kucoin_data_past_seven_days_df)
display(sum_all_data_df)


,Trading Pair,Order Type,Price,Quantity,Cost USDT,Fee USDT
0,BLOK-USDT,buy,0.649487,11206.304549,1039.503553,2.079007
1,DYDX-USDT,buy,110.600000,106.590400,1684.128320,1.684128
2,DYDX-USDT,sell,113.543000,100.000000,1622.028099,1.622028
3,HAKA-USDT,buy,8.369900,1284.291037,347.000000,0.694000
4,HOTCROSS-USDT,buy,0.445360,779.144961,347.000000,1.041000
5,KEEP-USDT,buy,1.428800,485.722284,347.000000,0.347000
6,LSS-USDT,buy,4.696780,221.637531,347.000000,0.694000
7,RMRK-USDT,buy,149.731100,129.937840,1618.383100,3.236766
8,RMRK-USDT,sell,65.000000,129.937800,1689.191400,3.378383
9,SKU-USDT,sell,9.450800,8000.000000,2434.792780,2.434793


In [75]:
unique_trading_pairs = []
unique_trading_pairs.append(kucoin_data_past_seven_days_df["Trading Pair"].unique())

functions_list.most_freq_trade_pair(kucoin_data_past_seven_days_df)

#display(unique_trading_pairs)
#display(unique_trading_pairs[0][0])

You traded 6.388908122967046 0.09234 times, in the last seven days.


In [78]:

rmrk_usdt = sum_all_data_df[sum_all_data_df["Trading Pair"]=="RMRK-USDT"]

rmrk_usdt_buy = rmrk_usdt[rmrk_usdt["Order Type"]=="buy"]
rmrk_usdt_buy_cost = rmrk_usdt_buy["Cost USDT"]
rmrk_usdt_buy_fee = rmrk_usdt_buy["Fee USDT"]
#rmrk_usdt_total_buy_expense = (rmrk_usdt_buy_cost + rmrk_usdt_buy_fee)
rmrk_usdt_total_buy_expense = np.array(rmrk_usdt_buy_cost + rmrk_usdt_buy_fee)

rmrk_usdt_sell = rmrk_usdt[rmrk_usdt["Order Type"]=="sell"]
rmrk_usdt_sell_cost = rmrk_usdt_sell["Cost USDT"]
rmrk_usdt_sell_fee = rmrk_usdt_sell["Fee USDT"]
rmrk_usdt_total_sell_revenue = rmrk_usdt_sell_cost - rmrk_usdt_sell_fee





#display(rmrk_usdt)
#
#display(rmrk_usdt_buy)
#display(rmrk_usdt_buy_fee)
#display(rmrk_usdt_buy_cost)
#display(rmrk_usdt_total_buy_expense)
#
#display(rmrk_usdt_sell)
#display(rmrk_usdt_sell_fee)
#display(rmrk_usdt_sell_cost)
#display(rmrk_usdt_total_sell_revenue)
print(rmrk_usdt_total_buy_expense)


[1621.6198661]


In [77]:

#trade_pair_grouped = sum_all_data_df["Trading Pair"] 

#def iterator():
#    for index in range(len(sum_all_data_df)):
#        if index in unique_trading_pairs:
#            var = sum_all_data_df[sum_all_data_df["Trading Pair"] == index]


#var_buy = var[var["Order Type"]=="buy"]
#var_sell =  var[var["Order Type"]=="sell"]
#
#var_cost_buy = var_buy["Cost USDT"]
#var_fee_buy = var_buy["Fee USDT"]
#
#var_income_sell = var_sell["Cost USDT"]
#var_fee_sell = var_sell["Fee USDT"]

#printdef (var)
#print(var_sell)
#print(var_buy)
#print(var_cost)
#print(var_fee)

#def total_expense_buy():
#    total_cost_buy = var_cost_buy + var_fee_buy
#    return total_cost_buy
#    print(total_cost_buy)
#
#def total_income_sell():
#    total_income_sell = var_income_sell - var_fee_sell
#    return total_income_sell
#    print(total_cost_sell)

#def check_value(x):
#    for i in list_CI:
#        if i.lower() in x.lower():
#           return i
#    return None
#
#df['new_col'] = df.apply(lambda x['CI Name']: check_value(x), axis = 1)



,Trading Pair,Order Type,Price,Quantity,Cost USDT,Fee USDT
0,BLOK-USDT,buy,0.649487,11206.304549,1039.503553,2.079007
1,DYDX-USDT,buy,110.600000,106.590400,1684.128320,1.684128
2,DYDX-USDT,sell,113.543000,100.000000,1622.028099,1.622028
3,HAKA-USDT,buy,8.369900,1284.291037,347.000000,0.694000
4,HOTCROSS-USDT,buy,0.445360,779.144961,347.000000,1.041000
5,KEEP-USDT,buy,1.428800,485.722284,347.000000,0.347000
6,LSS-USDT,buy,4.696780,221.637531,347.000000,0.694000
7,RMRK-USDT,buy,149.731100,129.937840,1618.383100,3.236766
8,RMRK-USDT,sell,65.000000,129.937800,1689.191400,3.378383
9,SKU-USDT,sell,9.450800,8000.000000,2434.792780,2.434793
